# 課題: コード生成

# 1. 与えられたコード / Given Code

* cc_ast.mll : 構文木定義
* cc_parse.mly : 構文定義 (ocamlyacc/menhirのコード)
* cc_lex.mll : 字句解析器 (ocamllexのコード)
* cc_cogen.ml : コード生成のスケルトン
* cc.mll : メイン関数

In [1]:
/* ----- 実行せよ / Execute this cell ----- */
//% cmd: ls

Makefile
cc.ml
cc_ast.ml
cc_cogen.ml
cc_lex.mll
cc_parse.mly
ex09_4_cc_cogen.c.ipynb
test.mk


# ビルド

In [2]:
/* ----- 実行せよ / Execute this cell ----- */
//% cmd: make

ocamlbuild -use-menhir cc.byte
ocamldep.opt -modules cc.ml > cc.ml.depends
ocamldep.opt -modules cc_cogen.ml > cc_cogen.ml.depends
ocamllex.opt -q cc_lex.mll
ocamldep.opt -modules cc_lex.ml > cc_lex.ml.depends
menhir --raw-depend --ocamldep 'ocamldep.opt -modules' cc_parse.mly > cc_parse.mly.depends
ocamldep.opt -modules cc_ast.ml > cc_ast.ml.depends
ocamlc.opt -c -o cc_ast.cmo cc_ast.ml
menhir --ocamlc ocamlc.opt --infer cc_parse.mly
+ menhir --ocamlc ocamlc.opt --infer cc_parse.mly
ocamldep.opt -modules cc_parse.mli > cc_parse.mli.depends
ocamlc.opt -c -o cc_parse.cmi cc_parse.mli
ocamlc.opt -c -o cc_cogen.cmo cc_cogen.ml
ocamlc.opt -c -o cc_lex.cmo cc_lex.ml
ocamlc.opt -c -o cc.cmo cc.ml
ocamldep.opt -modules cc_parse.ml > cc_parse.ml.depends
ocamlc.opt -c -o cc_parse.cmo cc_parse.ml
ocamlc.opt cc_cogen.cmo cc_ast.cmo cc_parse.cmo cc_lex.cmo cc.cmo -o cc.byte
echo  > cc.top
echo "ocaml -I _build _build/cc_ast.cmo _build/cc_parse.cmo _build/cc_lex.cmo _build/cc_cogen.cmo _build/cc.cm

# 再ビルド

In [593]:
/* ----- 実行せよ / Execute this cell ----- */
//% cmd: make clean

rm -rf _build cc.top cc.byte cc.native


In [594]:
/* ----- 実行せよ / Execute this cell ----- */
//% cmd: make

ocamlbuild -use-menhir cc.byte
ocamldep.opt -modules cc.ml > cc.ml.depends
ocamldep.opt -modules cc_cogen.ml > cc_cogen.ml.depends
ocamldep.opt -modules cc_ast.ml > cc_ast.ml.depends
ocamlc.opt -c -o cc_ast.cmo cc_ast.ml
ocamllex.opt -q cc_lex.mll
ocamldep.opt -modules cc_lex.ml > cc_lex.ml.depends
menhir --raw-depend --ocamldep 'ocamldep.opt -modules' cc_parse.mly > cc_parse.mly.depends
menhir --ocamlc ocamlc.opt --infer cc_parse.mly
+ menhir --ocamlc ocamlc.opt --infer cc_parse.mly
ocamldep.opt -modules cc_parse.mli > cc_parse.mli.depends
ocamlc.opt -c -o cc_parse.cmi cc_parse.mli
ocamlc.opt -c -o cc_cogen.cmo cc_cogen.ml
+ ocamlc.opt -c -o cc_cogen.cmo cc_cogen.ml
File "cc_cogen.ml", line 198, characters 16-505:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(EXPR_NUM _|EXPR_BIN_OP (_, _, _)|EXPR_UN_OP (_, _))
File "cc_cogen.ml", line 235, characters 4-2352:
Warning 8: this pattern-matching is not exhaustive.
Here is an example 

# 2. 実行

* そのまま実行すると, 例外が発生する

In [209]:
/* ----- 実行してエラーを見よ / Execute this cell and see an error ----- */
//% cmd: ./cc.byte ../test/fun.c fun.s

* 例外を発しているのは以下の cogen_program 関数

```
exception Not_implemented of string
;;
(* これを実装するのが役目 *)
let rec cogen_program program src_name =
  raise (Not_implemented
           "<not implemented> you should implement cogen_program in cc_cogen.ml\n")
;;
```
* この関数は program の構文木をアセンブリ言語に変換することを意図している

# 本番課題 1 

* cc_cogen.ml の cogen_program 関数の中身を実装して, program の構文木からアセンブリ言語の文字列を返すようにせよ

ヒント:

* 前問同様, 必要に応じて他の種類の構文木をアセンブリ言語に変換する関数も追加する
* 例えば cogen_definition, cogen_stmt, cogen_expr, などを作ることになるだろう

* テストは簡単なプログラムから始めるのが基本
* 簡単なテストプログラムが ../test/fun.c に多数あるので適宜利用して下さい
* 以下が実行できればコンパイル自身は出来ていることになる

In [595]:
/* ----- 以下で書いたコードをテストせよ / Test your code here ----- */
//% cmd: ./cc.byte ../test/fun.c fun.s

* 以下で, 生成されたコードが正しく動くか(同じコードをGCCでコンパイルしたときと, コンパイル後のプログラムの出力が同じ)

In [597]:
/* ----- 以下で書いたコードをテストせよ / Test your code here ----- */
//% cmd: make -f test.mk

make: Nothing to be done for 'all'.


# 発展課題 2 (オプション)

* mincコンパイラに拡張を施して, より広範囲のプログラムを扱えるようにせよ
* 何をしてもよいが, ありそうな候補は以下(以下は易しい順)
  * for文をサポートする
  * ポインタ型をサポートする
    * それに加え「型チェック」(異なる型の間の代入を禁止する)をサポートする
    * それに加え型の強制(キャスト)をサポートする
  * 浮動小数点数をサポートする
  * 構造体をサポートする